In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from iss_analysis.io import get_mcherry_cells
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from pathlib import Path

arial_font_path = "/nemo/lab/znamenskiyp/home/shared/resources/fonts/arial.ttf"  # update path as needed
arial_prop = fm.FontProperties(fname=arial_font_path)
plt.rcParams["font.family"] = arial_prop.get_name()
fm.fontManager.addfont(arial_font_path)
matplotlib.rcParams["pdf.fonttype"] = 42  # for pdfs
main_path = Path("/nemo/lab/znamenskiyp/")

project = 'becalia_rabies_barseq'
mouse = 'BRAC8498.3e'

# Get the good mCherry cells, i.e. the one for which we have masks and we match the 
# starters
mcherry_curated = get_mcherry_cells(
    project, mouse, verbose=True, which="curated", prefix="mCherry_1"
)
# For chamber6, detection fails because of high background, use the manual click instead
mcherry_manual =  get_mcherry_cells(
    project, mouse, verbose=True, which="manual", prefix="mCherry_1"
)

m6 = mcherry_manual.query("chamber == 'chamber_06'")
n6 = len(m6)
total = len(mcherry_curated) + n6

print(f"{n6} cells in chamber 6 out of {total}, that's {n6/total*100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt

# Plot manual for chamber 6
m6 = mcherry_manual.query("chamber == 'chamber_06'")
m6 = m6.groupby('roi').aggregate(len).x
plt.plot(m6.index.astype(int)-10, m6.values, 'o-')

# Plot curated for the other
n=10
for c in ['07','08','09','10']:
    df = mcherry_curated.query(f"chamber == 'chamber_{c}'")
    d = df.groupby('roi').aggregate(len).x
    plt.plot(d.index + n, d.values, 'o-')
    n += d.index.max()

plt.xlabel('Slice index')
plt.ylabel('# of mcherry cells')

In [ ]:
from brisc.manuscript_analysis import starter_cell_counting as sc_count
DATA_ROOT = Path("/nemo/lab/znamenskiyp")

In [ ]:
# Add the starter position
from iss_analysis.io import get_starter_cells, get_sections_info
import numpy as np
save_fig = True
save_path = main_path / "home/shared/presentations/becalick_2025"
fontsize_dict = {"title": 8, "label": 8, "tick": 6, "legend": 6}
cm = 1 / 2.54

sec_inf = get_sections_info(project, mouse, chamber=None)
starters_positions = get_starter_cells(project, mouse)
fig = plt.figure(figsize=(11.5 * cm, 5 * cm), dpi=600)
# Cell density vs PhP.eB dilution
ax_starter_density= fig.add_subplot(1,2,1)
im = sc_count.plot_starter_dilution_densities(
    ax_starter_density,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    processed=DATA_ROOT / "home/shared/projects",
)
ax_starter_density.set_yscale('log')
ax = fig.add_subplot(1,2,2)
axt = ax.twinx()
ax.plot((m6.index.astype(int)-10)*20, m6.values, 'o-',  ms=3,color='grey')
n = 10

for c in ['07','08','09','10']:
    df = mcherry_curated.query(f"chamber == 'chamber_{c}'")
    d = df.groupby('roi').aggregate(len).x
    ax.plot((d.index + n)*20, d.values, 'o-', ms=3,
            color='darkred', label='mCherry cells' if c=='10' else '__no_label__')
    sdf = starters_positions.query(f"chamber == 'chamber_{c}'")
    if len(sdf):
        sd = sdf.groupby('roi').aggregate(len).y
        axt.plot((sd.index + n)*20, sd.values, 's-',  ms=3, color='dodgerblue', label='Starter cells' if c=='10' else '__no_label__') 
    n += d.index.max()
axt.set_ylabel('# of starter cells', color='dodgerblue', fontsize=fontsize_dict['label'])
ax.set_yticks(np.arange(0,201,50))
ax.set_yticklabels(np.arange(0,201,50), color='darkred', fontsize=fontsize_dict['tick'])
ax.set_ylim(0, 200)
axt.set_yticks(np.arange(0, 81, 20))
axt.set_yticklabels(np.arange(0, 81, 20), color='dodgerblue', fontsize=fontsize_dict['tick'])
axt.set_ylim(0, 80)
ax.set_xlabel(r'Slice A/P position ($\mu m$)', fontsize=fontsize_dict['label'])
ax.set_ylabel('# of mCherry cells', color='darkred', fontsize=fontsize_dict['label'])
xt = np.arange(0, 1001, 500)
ax.set_xticks(xt, labels=xt, fontsize=fontsize_dict['tick'])
fig.tight_layout()

if save_fig:
    fig.savefig(save_path / "supplementary_figure_3a.pdf")

In [ ]:
import pandas as pd
project = "rabies_barcoding"
processed=Path("/nemo/lab/znamenskiyp/home/shared/projects/")
mouse_csv = processed / project / "mice_list.csv"
mice_df = pd.read_csv(mouse_csv, skipinitialspace=True)
mice_df.set_index("Mouse", inplace=True, drop=False)


In [ ]:
mice_df